In [3]:
#CVX

import create_data_upd as OD
import numpy as np
from sklearn.linear_model import OrthogonalMatchingPursuit
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)

# Define dimensions
n_features = 4000   # Number of columns in A
cs_ratio=0.25
n_samples = cs_ratio*n_features  # Number of rows in A (measurements)
k_sparse = 5  #precentage
len_test = 1000
rows = 400
cols = 10
sig1vt_supp=10
zt_noise_sigma=0.01
data = OD.Data(rows=rows, cols=cols, sig1vt_supp=sig1vt_supp, k_sparse=k_sparse,zt_noise_sigma=zt_noise_sigma)
X_dataset_test, C_dataset_test, Z_dataset_test = data.create_Dataset(len_test)

# Create a random measurement matrix A
A = np.random.randn(int(n_samples), n_features)

In [ ]:
import numpy as np
import cvxpy as cp


z_true = Z_dataset_test[0].flatten()
z_true=(z_true-z_true.mean())/z_true.std()


# Measurements
y_z = A @ z_true

# Define the variable
z_est = cp.Variable(n_features)

# Define the optimization problem
objective = cp.Minimize(cp.norm1(z_est))
constraints = [A @ z_est == y_z]
prob = cp.Problem(objective, constraints)

# Solve the problem
result = prob.solve()

# Output the results
print("Status:", prob.status)
print("Optimal value:", result)
print("Number of non-zero coefficients in x_true:", np.sum(z_true != 0))
print("Number of non-zero coefficients in x_estimated:", np.sum(np.abs(z_est.value) > 1e-4))

# Compute recovery error
recovery_error = np.linalg.norm(z_true - z_est.value) / np.linalg.norm(z_true)
print("Relative recovery error:", recovery_error)
